In [5]:
import pandas as pd
import numpy as np
import getpass
import json
import requests

1. Set up address screen call with `requests`
2. Parse out response of API response to csv columns
3. Load csv with addresses in first column
4. For each row in CSV, run Address Screening API
5. export results to CSV

`https://api.chainalysis.com/api/kyt/v1/users/{{userId}}/withdrawaladdresses`

In [6]:
##### ENTER YOUR API KEY
api_key = getpass.getpass("Enter your API Key:")

Enter your API Key: ································································


In [12]:
payload = {}

In [22]:
payload = {
    "network":"Ethereum"
    ,"asset":"ETH"
    ,"address":"0x92193107FB10B3B372AB21cC90b5a4DBd67861d9"}

In [23]:
headers = {
    'Token' : api_key
    ,'Content-Type':'application/json'
}

In [29]:
r = requests.post(
    url='https://api.chainalysis.com/api/kyt/v1/users/batch_user_id/withdrawaladdresses'
    ,data=payload
    ,headers=headers)

In [30]:
r

<Response [400]>

In [24]:
url = 'https://api.chainalysis.com/api/kyt/v1/users/batch_user_id/withdrawaladdresses'

In [25]:
response = requests.request("GET", url, headers=headers, data=payload)

In [26]:
response.json()

{'limit': 100, 'offset': 0, 'total': 0, 'data': []}

# Postman

In [31]:
import requests
import json

url = "https://api.chainalysis.com/api/kyt/v1/users/User 875/withdrawaladdresses"

payload = {
    "network":"Ethereum"
    ,"asset":"ETH"
    ,"address":"0x92193107FB10B3B372AB21cC90b5a4DBd67861d9"}

headers = {
  'token': '06094f6ef2909569d7533ecd8706cad35703709de49713299d6e74edb937b878',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


In [33]:
response

<Response [400]>

# Anand's

In [1]:
import requests
import json

url = "https://api.chainalysis.com/api/kyt/v1/users"

payload = ""
headers = {
  'token': '833beda07b3a458662e1dbeb877fc5d5883b0cb298f249e5b554f32b9264707c',
  'Content-Type': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)
print(response.json())

{'limit': 100, 'offset': 0, 'total': 62319, 'data': [{'userId': '00001bd2-cdb1-4707-b125-97dfdc46d3f4', 'score': 'green', 'lastActivity': '2013-09-19T11:46:31Z', 'scoreUpdatedDate': '2022-01-26T13:33:58.964220Z', 'riskScore': 'LOW'}, {'userId': '0001', 'score': None, 'lastActivity': None, 'scoreUpdatedDate': None, 'riskScore': None}, {'userId': '0003', 'score': None, 'lastActivity': None, 'scoreUpdatedDate': None, 'riskScore': None}, {'userId': '000fa2744d2fe03b9c3f7899b069eb2202cb21718bfb8bff64c2a2e1c283bd6c', 'score': 'green', 'lastActivity': '2018-10-23T14:44:11Z', 'scoreUpdatedDate': '2021-09-16T02:30:01.254526Z', 'riskScore': 'LOW'}, {'userId': '002d41d6-8eb1-4448-8e11-98f98d2ce796', 'score': 'green', 'lastActivity': '2021-04-12T18:44:16Z', 'scoreUpdatedDate': '2021-09-14T17:09:05.073296Z', 'riskScore': 'LOW'}, {'userId': '0043c2dd-ff86-4f0c-ad74-4b38483a7406', 'score': 'red', 'lastActivity': '2021-04-24T06:33:04Z', 'scoreUpdatedDate': '2022-01-23T07:27:39.490494Z', 'riskScore': '